In [1]:
import random
import math
import operator
import pandas as pd

In [21]:
# 定义数据结构
### 用户打标签记录：records 字典类型，保存了user对item的tag，即{userid: {item1:[tag1, tag2], ...}}
### 用户打过的标签：user_tags[u][t] 记录用户打过特定标签的次数
### 用户打过标签的商品：user_items[u][i] 记录用户打过特定商品的次数
### 打上某标签的商品：tag_items[t][i]  记录商品被打上该标签的次数
### 某标签使用过的用户：tags_users[t][u] 记录用户打过该标签的次数

#保存user对item的tag
records = {} 
# 用户标签&商品标签
user_tags = dict()
tag_items = dict()
user_items = dict()
# 训练集&测试集
train_data = dict()
test_data = dict()

In [22]:
# 定义加载数据函数  
def load_data(file_path):
    # 此函数用于加载数据，加载后的数据存入records字典中，格式为{userid: {item1:[tag1, tag2], ...}}
    print('数据开始加载...')
    df = pd.read_csv(file_path,sep='\t')
    for i in range(len(df)):
        uid = df['userID'][i]
        iid = df['bookmarkID'][i]
        tag = df['tagID'][i]
        # 当键不存在时，设置为默认值{}
        records.setdefault(uid,{})
        records[uid].setdefault(iid,[])
        records[uid][iid].append(tag)
    print('数据集大小为：%d'%len(df))
    print('设置tag的人数：%d'%len(records))
    print('数据加载完成\n')
    
# 将数据集拆分为训练集合测试集
def train_test_split(ratio,seed=100):
    random.seed(seed)
    for u in records.keys():
        for i in records[u].keys():
            #ratio 比例的数据设置为测试集
            if random.random()<ratio:
                test_data.setdefault(u,{})
                test_data[u].setdefault(i,[])
                for t in records[u][i]:
                    test_data[u][i].append(t)
            else:
                train_data.setdefault(u,{})
                train_data[u].setdefault(i,[])
                for t in records[u][i]:
                    train_data[u][i].append(t)
    print('训练样本集：%d, 测试样本集：%d'%(len(train_data),len(test_data)))

In [37]:
#设置矩阵 mat[index, item] = 1
def addValueToMat(mat,index,item,value=1):
    # 此函数统计index中item出现的次数，将index值和item出现的次数存储在变量为mat的字典中{index,{item,value}}
    if index not in mat:
        mat.setdefault(index,{})
        mat[index].setdefault(item,value)
    else:
        if item not in mat[index]:
            mat[index][item] = value
        else:
            mat[index][item] += value

In [42]:
# 使用训练集，初始化user_tags, tag_items, user_items
def initState():
    # 初始化函数，用来判断用户和tag；item和tag；用户和item一起出现的次数，并保存在相应的字典中
    records = train_data
    for u,i in records.items():
        for item,tags in records[u].items():
            for tag in tags:
                #用户和tag的关系
                addValueToMat(user_tags,u,tag,1)
                #items 和 tag 的关系
                addValueToMat(tag_items,tag,item,1) #注意此处 item 和 tag 顺序不一样的化表达的含义和数据结构是不一样的
                #用户和item的关系
                addValueToMat(user_items,u,item,1)
    print('user_tags,tag_items,user_items初始化完成')
    print('user_tags size:%d,tag_items size:%d,usr_items size:%d'%(len(user_tags),len(item_tags),len(user_items)))

In [43]:
# 对用户推荐top_N
def recommend(user,N):
    recommend_items = dict()
    # 此函数为user推荐排其未曾使用过的排名前N项的产品(item),以字典形式返回
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以商品被打上相同标签的次数 wti）之和
    tagged_items = user_items[user]
    for tag,wut in user_tags[user].items():
         #print(self.user_tags[user].items())
        for item,wti in tag_items[tag].items():
            if item in tagged_items:
                continue
            if item not in recommend_items:
                recommend_items[item] = wut*wti
            else:
                recommend_items[item] = recommend_items[item]+wut*wti
    return sorted(recommend_items.items(),key=operator.itemgetter(1),reverse=True)[0:N]

In [46]:
# 计算准去率&回归率
def precisionAndRecall(N):
    hit = 0 
    h_recall = 0
    h_precision = 0
    for user,items in test_data.items():
        if user not in train_data:
            continue
        rank = recommend(user,N)
        for item,rui in rank:
            if item in items:
                hit = hit+1
        h_recall = h_recall + len(items)
        h_precision = h_precision+N
    print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
    # 返回准确率 和 召回率
    return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))

# 使用测试集，对推荐结果进行评估
def testRecommend():
    print("推荐结果评估")
    print("%3s %10s %10s" % ('N',"精确率",'召回率'))
    for n in [5,10,20,40,60,80,100]:
        precision,recall = precisionAndRecall(n)
        print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))

In [47]:
load_data('C:/Users/18280/Desktop/RS/Recommended_System-master/L2/delicious-2k/user_taggedbookmarks-timestamps.dat')
train_test_split(0.2)
initState()
testRecommend()

数据开始加载...
数据集大小为：437593
设置tag的人数：1867
数据加载完成

训练样本集：1860, 测试样本集：1793
user_tags,tag_items,user_items初始化完成
user_tags size:1860,tag_items size:0,usr_items size:1860
推荐结果评估
  N        精确率        召回率
一共命中 74 个, 一共推荐 8930 个, 用户设置tag总数 20861 个
  5      0.829%      0.355%
一共命中 113 个, 一共推荐 17860 个, 用户设置tag总数 20861 个
 10      0.633%      0.542%
一共命中 183 个, 一共推荐 35720 个, 用户设置tag总数 20861 个
 20      0.512%      0.877%
一共命中 272 个, 一共推荐 71440 个, 用户设置tag总数 20861 个
 40      0.381%      1.304%
一共命中 341 个, 一共推荐 107160 个, 用户设置tag总数 20861 个
 60      0.318%      1.635%
一共命中 395 个, 一共推荐 142880 个, 用户设置tag总数 20861 个
 80      0.276%      1.893%
一共命中 443 个, 一共推荐 178600 个, 用户设置tag总数 20861 个
100      0.248%      2.124%
